In [38]:
import nltk
import math
from operator import eq 

def OpinosisGraph(Z, G, PRI):  #paper Alg1
    for i in range(len(Z)):  
        for j in range(len(Z[i])):  
            LABEL = Z[i][j]
            PID = j
            SID = i
            if LABEL in G:
                PRI[LABEL].append((SID, PID))
            else:
                G[LABEL] = list()
                PRI[LABEL] = [(SID, PID)]
            if j > 0 and LABEL not in G[Z[i][j-1]]:
                G[Z[i][j-1]].append(Z[i][j])
                
def VSN(node):   #paper def 1
    global PRI1
    sigma_vsn = 15
    ele = PRI1[node]
    sum = 0
    for i in range(len(ele)):
        sum = sum + ele[i][1]
    return (sum/len(ele)) <= sigma_vsn

def VEN(node):   #paper def 2
    prohibit = ['and', 'but', 'for', 'nor', 'or', 'so', 'yet', ',', '.', '!', '?']
    if(node[0] in prohibit):
        return True
    return False

def collapsible(node):  #paper def 5
    if(node[1] == 'VERB'):
        return True
    else: 
        return False

def valid_sent(sent):     #paper doesn't specify how to find a valid sentence in grammar
    prohibit = {'VERB':['VERB'],      #We check the part of speech of two adjacent words and prevent two same words appearing in one sentence
        'NOUN':['ADJ','PRON','NOUN'],
        'ADJ':['VERB','PRON','ADV'],
        'ADV':['ADJ','NOUN','PRON','ADV'],
        'PRON':['ADJ','PRON','NOUN'], #I,she
        'CONJ':['CONJ','PRT','ADP'], #and
        'PRT':['CONJ','PRT','ADP'], # 's
        '.':['VERB','.','ADJ','PRON','NOUN','ADV','PRON','CONJ','PRT','ADP','NUM','DET','X'], #punctuation
        'ADP':['CONJ','PRT','ADP'], #prep+介詞
        'NUM':[],
        'DET':['CONJ','PRT','ADP'], #to
        'X':[]
      }
    for i in range(0,len(sent)-1):
        if(sent[i+1][1] in prohibit[sent[i][1]]):
              return False
    buf = []
    for i in sent:
        if(i not in buf):
            buf.append(i)
        elif(i[1] == 'ADJ' or i[1] == 'ADV' or i[1] == 'NOUN'):
            return False
    return True

def check_inter(PRIa,PRIb):  #paper Alg3 line12
    PRIc = []
    sigma_gap = 4
    for first in PRIa:
        for second in PRIb:
            if(first[0] == second[0] and abs(first[1]-second[1]) <= sigma_gap):
                PRIc.append(second)
    return PRIc

def eliminate_dup(clist): #paper Alg3 line22
    buf = []
    for ele in clist:
        if ele[0] not in buf:
            buf.append(ele[0])
        else:
            clist.remove(ele)
    return clist

def common_elements(list1, list2):  #paper Alg3 line22
    count = 0
    for element in list1:
        if element in list2:
            count= count+1
    return count
                                            
def jaccard_fail(a,b):   #paper Alg2 line14: eliminate duplicates with jaccard similarity
    sigma_j = 0.3
    inter = common_elements(a,b)
    return ((inter/(len(a)+len(b)-inter)) >= sigma_j)
    

def eliminate_dup2(clist): #paper Alg2 line14: eliminate duplicates with jaccard similarity
    buf = []
    first = True
    for ele in clist:
        temp = []
        similar = False
        if(first):
            buf.append(ele)
            first = False
        else:
            for seen in buf:
                if jaccard_fail(seen[0],ele[0]):
                    similar = True
                    if ele[1]>seen[1]:                           
                        temp.append(seen)
            if len(temp) != 0:  
                for seen in temp:
                    buf.remove(seen)
                buf.append(ele)
            elif similar == False:
                buf.append(ele)  
    return buf



def find_avg_score(clist):    #paper Alg3 line23
    if len(clist)==0:
        return 0
    sum = 0
    for ele in clist:
        sum = sum + ele[1]
    return sum/len(clist)

def stitch(Canchor,CC):    #paper def 7,Alg3 line23
    if len(CC) == 0:
        return Canchor
    winner = CC[0]
    for ele in CC:
        if ele[1] > winner[1]:
            winner = ele
    ans = []
    return (Canchor+winner[0])

def find_sum(C):             #paper Alg2 line16,17
    winner = (None,-1)
    runner_up = (None,-1)
    for ele in C:
        if ele[1] > winner[1]:
            if winner[1] > runner_up[1]:
                runner_up = winner
            winner = ele
        elif ele[1] > runner_up[1]:
            runner_up = ele
    ans1 = ""
    for words in winner[0]:
        ans1 = ans1 + words[0] + " "
    ans1 = ans1[0].upper() + ans1[1:(len(ans1)-1)]
    ans2 = ""
    for words in runner_up[0]:
        ans2 = ans2 + words[0] + " "
    ans2 = ans2[0].upper() + ans2[1:(len(ans2)-1)]
    return ans1,ans2

def Traverse(clist,vk,score,PRI_overlap,sent,length):    #paper Alg3
    global G1
    global PRI1
    sigma_r = 2
    redundancy = len(PRI_overlap)
    if length>10:        #we add a path length limit here, so the program won't have recursion error
        return
    if (redundancy / length >= sigma_r):  
        if (VEN(vk)):
            if (valid_sent(sent)):
                finalscore = score/length
                clist.append((sent,finalscore))
                return 
        for vn in G1[vk]:
            PRI_new = PRI_overlap+check_inter(PRI_overlap,PRI1[vn])
            redundancy = len(PRI_new)
            new_sent = sent+[vn]
            L = length + 1
            new_score = score + redundancy  
            if (collapsible(vn)):
                Canchor = new_sent
                tmp = []
                for vx in G1[vn]:
                    Traverse(tmp,vx,0,PRI_new,[vx],L)
                    CC = eliminate_dup(tmp) 
                    avg_CC_score = find_avg_score(CC)
                    finalscore = new_score + avg_CC_score
                    stitch_sent = stitch(Canchor,CC)  
                    if(valid_sent(stitch_sent)):
                        clist.append((stitch_sent,finalscore))
            else:
                Traverse(clist,vn,new_score,PRI_new,new_sent,L)

def first_lower(s):     #string processing
    if s[0] == "I":
        return s
    else:
        return s[0].lower() + s[1:]
        
def read_text(file_name):    #readfile
    redundant = '<br />'
    review = []
    with open(file_name,'r') as file:
        for article in file:
            article = article.replace(redundant," ")
            sentences = nltk.sent_tokenize(article)    
            for sentence in sentences:
                sentence = first_lower(sentence)
                review.append(nltk.pos_tag(nltk.word_tokenize(sentence),tagset='universal'))           
    file.close()
    return review



In [39]:
import nltk
import os
filenames = [f for f in os.listdir('.') if os.path.isfile(f)]
for name in filenames:         #read all files in the folder
    a,b = name.split('.')
    if b == 'txt':
        print(name)
        Z1 = read_text(name)
        G1 = dict()
        PRI1 = dict()
        OpinosisGraph(Z1, G1, PRI1)   #paper Alg2
        candidates = []
        for keys in G1:
            if(VSN(keys)):
                clist = []
                sum_sent = [keys]
                score = 0
                Traverse(clist,keys,score,PRI1[keys],sum_sent,1)
                candidates = candidates + clist
        C = eliminate_dup2(candidates)
        summary = find_sum(C)
        print("summary1: ",summary[0])
        print("summary2: ",summary[1])

00_9.txt
summary1:  The pettiness of the episode in which a student repeatedly tried
summary2:  In the episode in which a student repeatedly tried to burn
01_7.txt
summary1:  The teachers of the small adventures of the cast is
summary2:  The cast is nearly a similar format about the small stories going
02_9.txt
summary1:  The escapades of a superbly caricatured
summary2:  The show does n't shy
03_10.txt
summary1:  Have to the story to you have to you have to you have
summary2:  The story to the world 's a grand experiment that said
04_10.txt
summary1:  Is in top form in the famous in-the-dark scene .
summary2:  In top form in the target of evil guys and
05_10.txt
summary1:  In the middle of the score of the positive side regarding
summary2:  Of the middle of the score of the positive side regarding
06_7.txt
summary1:  The best part of this movie is the other comments focus
summary2:  N't understand why the other comments focus on McConaughey .
07_7.txt
summary1:  N't like Stanley & Iri

In [ ]:
'''
problem:
1.We incorrectly calculated new PRI by obtaining the union of the two sets, but the paper's method is finding the intersection of two sets
2.Performance for some cases is terrible(only a meaningless word is found:08_7.txt,11_9.txt) 
3.Repetitive rate of two summaries is high for some cases(05_10.txt,16_9.txt)
4.The grammar of each sentence is problematic because we don't know how to define a valid sentence
'''